In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/content/tesla_stock_data_final_cleaneddata(noduplciates_nomissingvalues).csv')

# Convert 'timestamp' to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Check for missing values
print(f"Missing values:\n{df.isnull().sum()}")




Missing values:
close          0
high           0
low            0
trade_count    0
open           0
volume         0
vwap           0
dtype: int64


In [ ]:
# Date-based features
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['year'] = df.index.year
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)  # Weekend feature

# Rolling statistics (7-day and 30-day averages)
df['7_day_avg'] = df['close'].rolling(window=7).mean()
df['30_day_avg'] = df['close'].rolling(window=30).mean()

# Volatility (difference between high and low prices over 7-day window)
df['7_day_volatility'] = df['high'].rolling(window=7).max() - df['low'].rolling(window=7).min()

# Lag features (previous day, 5 days ago, 10 days ago)
df['close_lag_1'] = df['close'].shift(1)
df['close_lag_5'] = df['close'].shift(5)
df['close_lag_10'] = df['close'].shift(10)

# Remove rows with NaN values generated by rolling and lag features
df.dropna(inplace=True)

# Function to create the target high and low prices
def create_targets(df, n_days=10):
    df['target_high'] = df['high'].shift(-n_days).rolling(window=n_days).max()
    df['target_low'] = df['low'].shift(-n_days).rolling(window=n_days).min()
    df = df.dropna(subset=['target_high', 'target_low'])
    return df

df = create_targets(df, n_days=10)

# Feature columns and target variables
X = df[['close', '7_day_avg', '30_day_avg', '7_day_volatility', 'close_lag_1', 'close_lag_5', 'close_lag_10', 'day_of_week', 'month', 'is_weekend']]
y_high = df['target_high']
y_low = df['target_low']

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scale features for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(df[['close', '7_day_avg', '30_day_avg', '7_day_volatility', 'close_lag_1', 'close_lag_5', 'close_lag_10', 'day_of_week', 'month', 'is_weekend']])

# Optimized LSTM data preparation
def prepare_lstm_data_optimized(df, features, target_col, sequence_length=30):
    data = df[features].values
    target = df[target_col].values
    num_samples = len(data) - sequence_length

    # Pre-allocate memory for X and y
    X = np.zeros((num_samples, sequence_length, len(features)))
    y = np.zeros(num_samples)

    for i in range(sequence_length, len(data)):
        X[i-sequence_length] = data[i-sequence_length:i]
        y[i-sequence_length] = target[i]

    return X, y

# Prepare data for LSTM
X_lstm_high, y_lstm_high = prepare_lstm_data_optimized(df, df[['close', '7_day_avg', '30_day_avg', '7_day_volatility', 'close_lag_1', 'close_lag_5', 'close_lag_10', 'day_of_week', 'month', 'is_weekend']].columns, 'target_high', sequence_length=10)
X_lstm_low, y_lstm_low = prepare_lstm_data_optimized(df, df[['close', '7_day_avg', '30_day_avg', '7_day_volatility', 'close_lag_1', 'close_lag_5', 'close_lag_10', 'day_of_week', 'month', 'is_weekend']].columns, 'target_low', sequence_length=10)

# Train-Test Split for LSTM
X_train_lstm, X_test_lstm, y_train_lstm_high, y_test_lstm_high = train_test_split(X_lstm_high, y_lstm_high, test_size=0.2, shuffle=False)
X_train_lstm, X_test_lstm, y_train_lstm_low, y_test_lstm_low = train_test_split(X_lstm_low, y_lstm_low, test_size=0.2, shuffle=False)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Build smaller LSTM model for High Price
lstm_model_high = Sequential()
lstm_model_high.add(LSTM(units=30, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))  # Reduced units from 50 to 30
lstm_model_high.add(Dropout(0.2))  # Keep dropout for regularization
lstm_model_high.add(LSTM(units=30, return_sequences=False))  # Reduced units from 50 to 30
lstm_model_high.add(Dropout(0.2))
lstm_model_high.add(Dense(units=1))  # Output layer for price prediction
lstm_model_high.compile(optimizer='adam', loss='mean_squared_error')

# Train the smaller LSTM model for High Price
lstm_model_high.fit(X_train_lstm, y_train_lstm_high, epochs=1, batch_size=16)  # Reduced epochs from 1 to 5 for more stable training

# Build smaller LSTM model for Low Price
lstm_model_low = Sequential()
lstm_model_low.add(LSTM(units=30, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))  # Reduced units from 50 to 30
lstm_model_low.add(Dropout(0.2))  # Keep dropout for regularization
lstm_model_low.add(LSTM(units=30, return_sequences=False))  # Reduced units from 50 to 30
lstm_model_low.add(Dropout(0.2))
lstm_model_low.add(Dense(units=1))  # Output layer for price prediction
lstm_model_low.compile(optimizer='adam', loss='mean_squared_error')

# Train the smaller LSTM model for Low Price
lstm_model_low.fit(X_train_lstm, y_train_lstm_low, epochs=1, batch_size=16)  # Reduced epochs from 1 to 5 for more stable training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


74595/74595 ━━━━━━━━━━━━━━━━━━━━ 438s 6ms/step - loss: 158353.9531
74595/74595 ━━━━━━━━━━━━━━━━━━━━ 426s 6ms/step - loss: 132767.9688


In [ ]:
print(X_test_lstm.shape)


(298380, 10, 10)


In [ ]:
y_pred_high_lstm = lstm_model_high.predict(X_test_lstm, batch_size=8)

37298/37298 ━━━━━━━━━━━━━━━━━━━━ 74s 2ms/step


In [ ]:
y_pred_low_lstm = lstm_model_low.predict(X_test_lstm, batch_size=8)

37298/37298 ━━━━━━━━━━━━━━━━━━━━ 75s 2ms/step
